In [30]:
import QuantLib as ql
import pandas as pd

### 1. DayCounter

- 이자 일수에 대해서 계산방식은 국가와 시장마다 적용하는 방법들이 다르다. 
- 이자 일수가 1년 미만인 경우 
  - 금리의 고시 단위가 연율이기 때문에 시간프레임을 연단위로 맞춘다.
  - 이러한 과정에서 필요한 것이 이자 일수 계산방식이다. 1년을 며칠로 볼것인지, 한달을 며칠로로 볼것인지에 대해서 선택해야한다.
  - 1년을 며칠로 볼것인지에 대해서는 ACT, 252, 360, 365일이 있다. ACT의 경우에 윤년이면 366일이 되고 나머지 252, 360, 365일은 고정해서 사용한다.
  - 한 달을 며칠로 볼것인지에 대해서는 ACT, 30, 영업일이 있다. ACT는 실제 이자구간의 일수를 계산하는 것이며 영업일은 휴일을 제외한 영업일만 카운트한다는 의미이다. 
  - 일반적으로 표기법은 $\frac{한달 관행}{일년 관행}$으로 사용하고 미국은 $\frac{ACT}{360}$, 유럽은 $\frac{30}{360}$, 한국은 $\frac{ACT}{365}$를 사용한다.

In [6]:
act360 = ql.Actual360() #ACT/360
act365 = ql.Actual365Fixed() # ACT/ 365
actact = ql.ActualActual() #ACT/ACT
thirty360 = ql.Thirty360() #30/360
b252 = ql.Business252() #BusinessDay/252

In [16]:
start_date = ql.Date(1, 1, 2020)
end_date = ql.Date(3, 6, 2020)

print(act360.dayCount(start_date, end_date))
print(act365.dayCount(start_date, end_date))
print(actact.dayCount(start_date, end_date))
print(thirty360.dayCount(start_date, end_date))
print(b252.dayCount(start_date, end_date))

154
154
154
152
104


- yearFunction()은 이자 일수 계산방식이 두 날짜 사이의 기간에서 연단위로 계산되서 얼마나 되는지 나타낸다.

In [18]:
print(act360.yearFraction(start_date, end_date))
print(act365.yearFraction(start_date, end_date))
print(actact.yearFraction(start_date, end_date))
print(thirty360.yearFraction(start_date, end_date))
print(b252.yearFraction(start_date, end_date))

0.42777777777777776
0.42191780821917807
0.4207650273224044
0.4222222222222222
0.4126984126984127


### 2. Schedule
- 채권, 스왑 등과 같이 금리를 기초하는 FICC 상품들은 이자지급 스케줄이 존재한다. 이자지급 스케줄이란 주기적으로 이자가 발생하는 것에 대한 구체적인 일련의 날짜들을 의미한다.
- ql.Schedule에는 효력발생일, 만기일, 이자지급주기, 달력, 이자결제일의 영업일관행, 만기일의 영업일관행, 날짜 생성 방식, 월말기준으로 이루어져있다.
- DateGeneration(날짜생성방식)은 다음과 같이 이루어져있다.
   - ql.Backward : 만기일로부터 효력발생일까지 후진방식으로 이자지급 스케줄 생성
   - ql.Forward : 효력발생일부터 만기일까지 전진 방식으로 이자지급 스케줄 생성
   - ql.Zero : 효력발생일과 만기일 사이에 어떤 결제일도 존재하지 않음.
   - ql.ThirdWednesday :효력발생일과 만기일을 제외한 모든 중간 이자지급일을 해당 월의 세 번째 수요일로 지정.
   - ql.Twentieth : 효력발생일을 제외한 모든 이자지급일을 해당월의 20일로 지정
   - ql.TwentiethlMM : 효력발생일을 제외한 모든 이자지급일을 3,6,9,12월 20일로 지정
- endOfMonth(월말기준)이 True이고 효력발생일이 월말 영업일이라면 스케줄을 생성할때 이자지급일을 월말 영업일로 맞추도록 조정한다. 이 값이 False이면 효력발생일이 어떤날인가에 상관없이 일반적인 일수계산방식을 사용한다.

In [26]:
effectiveDate = ql.Date(13, 4, 2020)
maturityDate = ql.Date(15, 4, 2023)
tenor = ql.Period(3, ql.Months) #이자지급주기
calender = ql.SouthKorea()
convention = ql.ModifiedFollowing
rule = ql.DateGeneration.Backward
endOfMonth = False

schedule = ql.Schedule(effectiveDate,  #효력발생일
                      maturityDate,    #만기일
                      tenor,           #이자지급주기
                      calender,        #켈린더
                      convention,      #이자일의 영업일관행
                      convention,      #만기일의 영업일관행
                      rule,            #날짜생성방식(후진방식 이자지급 스케줄)
                      endOfMonth)      #월말기준
schedule

<QuantLib.QuantLib.Schedule; proxy of <Swig Object of type 'Schedule *' at 0x0000024A53D6AE70> >

- schedule은 우리가 원하는 시장관행들을 바탕으로 이자지급 스케줄을 생성해 금융상품객체에 넘겨주는 역할은 한다. 즉, 미국 국채를 구현한다고 했을때 미국 국채라는 금융상품을 만드는 과정에서 이자지급 스케줄이 필요하고 이것을 schedule 클래스가 수행한다.

In [28]:
ref_date = ql.Date(4, 10, 2021)

print("Next payment date from {} : {}".format(ref_date, schedule.nextDate(ref_date)))
print("previous payment date from {} : {}".format(ref_date, schedule.previousDate(ref_date)))

Next payment date from October 4th, 2021 : October 15th, 2021
previous payment date from October 4th, 2021 : July 15th, 2021


In [31]:
pd.DataFrame({'date': list(schedule)})

,date
0,"April 13th, 2020"
1,"April 16th, 2020"
2,"July 15th, 2020"
3,"October 15th, 2020"
4,"January 15th, 2021"
5,"April 15th, 2021"
6,"July 15th, 2021"
7,"October 15th, 2021"
8,"January 17th, 2022"
9,"April 15th, 2022"


### 3. Quote 
- 파생상품에는 주가, 금리, 환율, 변동성 등 기초가되는 자산들이 필요하다. quote클래스는 실수값을 받아 기초자산의 객체를 만들어준다. 여기서는 SimpleQuote를 살펴보자.

In [39]:
quote = ql.SimpleQuote(3213.5342)
quote.value()

3213.5342

In [36]:
quote.setV

<QuantLib.QuantLib.ActualActual; proxy of <Swig Object of type 'QuantLib::ActualActual *' at 0x0000024A5684A3F0> >